# Cortisol Analysis

In [ ]:
from pathlib import Path
import json

import pandas as pd
import numpy as np
import pingouin as pg

import biopsykit as bp
from biopsykit.stats import StatsPipeline
from biopsykit.io import load_long_format_csv
from biopsykit.utils.dataframe_handling import multi_xs

from fau_colors import cmaps, register_fausans_font

import matplotlib.pyplot as plt
import seaborn as sns

from carwatch_analysis.io import load_cortisol_samples_reporting_times
from carwatch_analysis.utils import describe_groups_df
from carwatch_analysis.stats import create_unique_night_id
from carwatch_analysis.plotting import multi_paired_plot_auc, paired_plot_auc
from carwatch_analysis.data_processing.saliva import compute_auc_increasing


%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)

theme_kwargs = {"context": "talk", "style": "ticks", "palette": palette}
theme_kwargs_grid = {"context": "talk", "style": "ticks", "palette": palette, "font_scale": 0.8}
sns.set_theme(**theme_kwargs)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

export = True

pg.options["round"] = 4

palette

## Setup Paths

In [ ]:
base_path = Path("../..")
export_path = base_path.joinpath("exports")
result_path = base_path.joinpath("results")
stats_path = result_path.joinpath("statistics")

img_path = result_path.joinpath("plots")

paper_path = Path(json.load(Path("../paper_path.json").open(encoding="utf-8"))["paper_path"])
paper_img_path = paper_path.joinpath("img")

bp.utils.file_handling.mkdirs([result_path, stats_path, img_path, paper_img_path])

## Load Data

In [ ]:
cort_features = load_long_format_csv(export_path.joinpath("cortisol_features_processed_all_reporting_types.csv"))
cort_features = create_unique_night_id(cort_features)
cort_features.head()

In [ ]:
# Don't consider IMU and IMU_App because the cortisol features are the same as
# Selfreport and App since only the wake onset differs, but not the sampling times
reporting_types_fine = [
    "Naive",
    "AW & ST: Selfreport (without App)",
    "AW & ST: Selfreport (with App)",
    "AW & ST: App",
    "AW: Sensor, ST: Selfreport (without App)",
    "AW: Sensor, ST: Selfreport (with App)",
    "AW: Sensor, ST: App",
]
reporting_types_coarse = [
    "Naive",
    "AW & ST: Selfreport",
    "AW & ST: App",
    "AW: Sensor, ST: Selfreport",
    "AW: Sensor, ST: App",
]

reporting_types_fine_rename = [s.replace("without", "w/o").replace("with", "w/") for s in reporting_types_fine]
rename_mapper_reporting_types = dict(zip(reporting_types_fine, reporting_types_fine_rename))

reporting_types_fine = reporting_types_fine_rename

cort_features = cort_features.rename(rename_mapper_reporting_types, level="reporting_type")

In [ ]:
dict_saliva_export = {}

## S1

### Data Selection

In [ ]:
cort_features_analysis = cort_features.copy()
cort_features_analysis = cort_features_analysis.xs("ini_val", level="saliva_feature", drop_level=False)
cort_features_analysis = cort_features_analysis.reindex(reporting_types_fine, level="reporting_type")
cort_features_analysis.head()

In [ ]:
fig, ax = plt.subplots()

sns.boxplot(
    data=cort_features_analysis.reset_index(), x="reporting_type", y="cortisol", palette=cmaps.faculties_light, ax=ax
)
fig.tight_layout()

## $AUC_I$

### Data Selection

In [ ]:
cort_features_analysis = cort_features.copy()
cort_features_analysis = cort_features_analysis.xs("auc_i", level="saliva_feature", drop_level=False)
cort_features_analysis = cort_features_analysis.reindex(reporting_types_fine, level="reporting_type")
cort_features_analysis.head()

### Stats

In [ ]:
reporting_types = ["Naive", "AW & ST: Selfreport (w/ App)", "AW & ST: App"]

data_analysis = cort_features_analysis.reindex(reporting_types, level="reporting_type")

pipeline = StatsPipeline(
    steps=[("prep", "normality"), ("prep", "equal_var"), ("test", "rm_anova"), ("posthoc", "pairwise_tests")],
    params={
        "dv": "cortisol",
        "within": "reporting_type",
        "subject": "night_id",
        "multicomp": {"method": "bonf", "levels": False},
    },
)

pipeline.apply(data_analysis)
pipeline.export_statistics(stats_path.joinpath("stats_auci_reporting_type.xlsx"))
pipeline.display_results()

### Boxplots

In [ ]:
box_pairs, pvalues = pipeline.sig_brackets("posthoc", stats_effect_type="within", plot_type="single")

fig, ax = plt.subplots(figsize=(10, 5))

bp.protocols.plotting.saliva_feature_boxplot(
    data=data_analysis,
    x="reporting_type",
    saliva_type="cortisol",
    feature="auc_i",
    order=reporting_types,
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues, "verbose": 0},
    palette=cmaps.faculties_light,
    ax=ax,
)
ax.set_title("$AUC_I$", pad=12)
ax.set_xlabel("Log Type")
ax.set_ylim(ax.get_ylim()[0], ax.get_ylim()[1] * 1.05)

fig.tight_layout()

for path in [img_path]:
    bp.utils.file_handling.export_figure(fig, "img_boxplot_auci", path, ["pdf", "png"], dpi=300)

### Paired Plots

In [ ]:
data_increase = compute_auc_increasing(cort_features_analysis.reindex(reporting_types, level="reporting_type"), "auc_i")
dict_saliva_export["AUC_I_reporting_Type_Increase"] = data_increase
data_increase

In [ ]:
fig, ax = paired_plot_auc(
    data=cort_features_analysis, saliva_feature="auc_i", reporting_types=reporting_types, figsize=(6, 5)
)

In [ ]:
reporting_types = ["Naive", "AW & ST: Selfreport (w/ App)", "AW & ST: App"]

fig, axs = multi_paired_plot_auc(
    data=cort_features_analysis, saliva_feature="auc_i", reporting_types=reporting_types, figsize=(12, 5)
)

for path in [img_path, paper_img_path]:
    bp.utils.file_handling.export_figure(fig, "img_pairedplot_auci", path, ["pdf", "png"], dpi=300)

## S1 and $AUC_I$ in Table

In [ ]:
saliva_features = ["ini_val", "auc_i"]

s1_auc_result = cort_features.reindex(saliva_features, level="saliva_feature")
s1_auc_result = s1_auc_result.groupby(["reporting_type", "saliva_feature"]).agg(["mean", "std"])
s1_auc_result = s1_auc_result["cortisol"].unstack("saliva_feature")
s1_auc_result = s1_auc_result.swaplevel(0, 1, axis=1).sort_index(axis=1)
s1_auc_result = s1_auc_result.reindex(reporting_types_fine, level="reporting_type")
s1_auc_result = s1_auc_result.reindex(saliva_features, level="saliva_feature", axis=1)
s1_auc_result.round(2)

## Export

In [ ]:
bp.io.write_pandas_dict_excel(dict_saliva_export, result_path.joinpath("saliva_results.xlsx"))